In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import pandas as pd
import os
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:

application_df.drop(columns=['EIN', 'NAME'], inplace=True)
app_count = application_df['APPLICATION_TYPE'].value_counts()

replace_application = list(app_count[app_count < 500].index)

for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")

class_count = application_df['CLASSIFICATION'].value_counts()
replace_class = list(class_count[class_count < 1800].index)

for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")

application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [3]:
# create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

C:\Users\Admin\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [4]:
# merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)

C:\Users\Admin\AppData\Local\Temp\ipykernel_44044\838529343.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df = application_df.drop(application_cat,1)


In [5]:
# split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL.values
X = application_df.drop(columns=["IS_SUCCESSFUL"]).values

# split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)


# create a StandardScaler instances
scaler = StandardScaler()

# fit the StandardScaler
X_scaler = scaler.fit(X_train)

# scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [6]:
# define the checkpoint path and filenames
os.makedirs("checkpoints_opt/",exist_ok=True)
checkpoint_path = "checkpoints_opt/weights.{epoch:02d}.hdf5"

In [7]:
# define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# first hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, 
                          input_dim=number_input_features, 
                          activation="relu"))

# second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [8]:
# compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq="epoch")

# train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50,callbacks=[cp_callback])

# evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
774/804 [===========================>..] - ETA: 0s - loss: 0.5699 - accuracy: 0.7214
Epoch 1: saving model to checkpoints_opt\weights.01.hdf5
804/804 [==============================] - 1s 874us/step - loss: 0.5703 - accuracy: 0.7209
Epoch 2/50
757/804 [===========================>..] - ETA: 0s - loss: 0.5532 - accuracy: 0.7322
Epoch 2: saving model to checkpoints_opt\weights.02.hdf5
804/804 [==============================] - 1s 813us/step - loss: 0.5534 - accuracy: 0.7311
Epoch 3/50
756/804 [===========================>..] - ETA: 0s - loss: 0.5489 - accuracy: 0.7332
Epoch 3: saving model to checkpoints_opt\weights.03.hdf5
804/804 [==============================] - 1s 817us/step - loss: 0.5507 - accuracy: 0.7317
Epoch 4/50
755/804 [===========================>..] - ETA: 0s - loss: 0.5492 - accuracy: 0.7329
Epoch 4: saving model to checkpoints_opt\weights.04.hdf5
804/804 [==============================] - 1s 817us/step - loss: 0.5495 - accuracy: 0.7332
Epoch 5/50
766/804 [====

In [9]:
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 60
hidden_nodes_layer2 = 40



nn_new = tf.keras.models.Sequential()

# first hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# third hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="linear"))

# output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

# evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 1s 816us/step - loss: 0.5374 - accuracy: 0.7401
Epoch 2/100
804/804 [==============================] - 1s 794us/step - loss: 0.5375 - accuracy: 0.7385
Epoch 3/100
804/804 [==============================] - 1s 782us/step - loss: 0.5373 - accuracy: 0.7403
Epoch 4/100
804/804 [==============================] - 1s 790us/step - loss: 0.5373 - accuracy: 0.7400
Epoch 5/100
804/804 [==============================] - 1s 782us/step - loss: 0.5373 - accuracy: 0.7400
Epoch 6/100
804/804 [==============================] - 1s 795us/step - loss: 0.5373 - accuracy: 0.7395
Epoch 7/100
804/804 [==============================] - 1s 781us/step - loss: 0.5373 - accuracy: 0.7405
Epoch 8/100
804/804 [==============================] - 1s 835us/step - loss: 0.5369 - accuracy: 0.7406
Epoch 9/100
804/804 [==============================] - 1s 786us/step - loss: 0.5363 - accuracy: 0.7404
Epoch 10/100
804/804 [==============================] - 1s 791us/step - l

In [10]:
nn_new.save("AlphabetSoupCharity_Optimization.h5")